In [1]:
import pandas as pd
import quandl
import cred
import datetime
from ipywidgets import IntProgress
from IPython.display import display

In [2]:
quandl.ApiConfig.api_key = cred.quandl_api
 
def quandl_stocks(symbol, start_date=(2019, 1, 1), end_date=None, mod='WIKI'):
    """
    symbol is a string representing a stock symbol, e.g. 'AAPL'
 
    start_date and end_date are tuples of integers representing the year, month,
    and day
 
    end_date defaults to the current date when None
    """
    query=mod+'/'+symbol
 
    if not end_date: 
        end_date = datetime.date.today()
    else:
        try:
            end_date = datetime.date(*end_date) # if not in timestamp
        except:
            end_date = end_date
    return quandl.get(query, 
            returns='pandas', 
            start_date=start_date,
            end_date=end_date,
            collapse='daily',
            order='asc'
            )

In [3]:
# Getting a list of the securities details with the code
df_sec_list=pd.read_excel("Data/BSE Meta Data/Final_BSE_List.xlsx",sheets="Final")

In [6]:
# Loading in the already available historical data
%time
df_historical_eod=pd.read_pickle("Data/historical_bse.pkl")

Wall time: 0 ns


In [7]:
# Make the data into a multi-index and get the last index for the securities.
df_temp=df_historical_eod.reset_index().set_index(['Date','code'])
df_temp=df_temp.groupby(level='code').tail(1).reset_index()
df_temp['Date']=pd.to_datetime(df_temp['Date'])
df_temp=df_temp[df_temp['Date']>pd.Timestamp('2019/1/1')]
sym_date=dict(zip(df_temp['code'],df_temp['Date']))

#### Quandl Limit
Quandl has a API limit of 2000 calls per 10 minutes, so I split it out into 2 calls

The right way to do this is to measure the time it takes to get back each call, then dynamically allocate wait time in the loop. So the average wait time will be dynamically calculated. But not just KISS

In [8]:
%time
quandl_limit = 1500

f = IntProgress(min=0, max=quandl_limit) # instantiate the bar
display(f) # display the bar

df_new_eod=pd.DataFrame()

for sym,last_date in {k: sym_date[k] for k in list(sym_date)[:quandl_limit]}.items():
    f.value += 1
    df_eod=quandl_stocks(symbol=sym,start_date=last_date,mod="BSE")
    df_eod['code']=sym
    if(df_new_eod.empty):
        df_new_eod=df_eod
        print("---",sym)
        print(len(df_new_eod))
    else:
        df_new_eod=df_new_eod.append(df_eod)
        print("---",sym)
        print(len(df_new_eod))

Wall time: 0 ns


IntProgress(value=0, max=1500)

--- BOM500002
42
--- BOM507685
83
--- BOM500003
125
--- BOM500325
166
--- BOM500008
208
--- BOM500009
249
--- BOM532648
290
--- BOM500010
331
--- BOM500180
372
--- BOM500012
413
--- BOM500570
454
--- BOM500013
495
--- BOM532500
536
--- BOM500014
578
--- BOM500209
619
--- BOM500020
660
--- BOM511072
701
--- BOM500023
723
--- BOM500087
764
--- BOM500024
805
--- BOM500027
846
--- BOM532540
887
--- BOM500028
928
--- BOM500510
969
--- BOM500029
1010
--- BOM500112
1051
--- BOM500139
1059
--- BOM500142
1064
--- BOM500202
1067
--- BOM500322
1072
--- BOM500371
1075
--- BOM500458
1076
--- BOM501151
1077
--- BOM503635
1078
--- BOM503637
1079
--- BOM503669
1080
--- BOM504369
1082
--- BOM504701
1083
--- BOM506003
1086
--- BOM506016
1087
--- BOM507649
1088
--- BOM509069
1091
--- BOM511609
1102
--- BOM511642
1103
--- BOM512405
1106
--- BOM513418
1114
--- BOM513422
1115
--- BOM517556
1124
--- BOM519570
1130
--- BOM520077
1131
--- BOM521030
1132
--- BOM521038
1133
--- BOM521048
1134
--- BOM523724
1137


--- BOM500429
14689
--- BOM500459
14730
--- BOM500439
14764
--- BOM511505
14796
--- BOM500444
14837
--- BOM500830
14878
--- BOM500449
14919
--- BOM530239
14960
--- BOM500450
14977
--- BOM500456
15018
--- BOM532162
15059
--- BOM500460
15100
--- BOM500463
15125
--- BOM500464
15166
--- BOM524735
15207
--- BOM500467
15248
--- BOM532779
15289
--- BOM506590
15330
--- BOM532509
15371
--- BOM500790
15412
--- BOM500483
15453
--- BOM500488
15494
--- BOM504966
15535
--- BOM500490
15576
--- BOM500690
15617
--- BOM522108
15658
--- BOM500500
15699
--- BOM524230
15740
--- BOM500520
15781
--- BOM532178
15822
--- BOM533023
15863
--- BOM500540
15891
--- BOM500840
15932
--- BOM532497
15973
--- BOM500620
16014
--- BOM523642
16055
--- BOM512573
16096
--- BOM500645
16137
--- BOM500650
16178
--- BOM524000
16219
--- BOM500655
16260
--- BOM500660
16301
--- BOM532345
16342
--- BOM500672
16383
--- BOM503310
16424
--- BOM500674
16465
--- BOM526227
16506
--- BOM500676
16547
--- BOM523395
16588
--- BOM500680
16629


--- BOM506142
29351
--- BOM507753
29392
--- BOM506146
29433
--- BOM514043
29474
--- BOM506159
29475
--- BOM532885
29516
--- BOM506854
29557
--- BOM506180
29558
--- BOM506184
29568
--- BOM506186
29598
--- BOM532663
29639
--- BOM506687
29680
--- BOM506194
29721
--- BOM506222
29760
--- BOM526951
29801
--- BOM506235
29842
--- BOM524558
29883
--- BOM506248
29924
--- BOM531633
29965
--- BOM506260
30006
--- BOM506261
30047
--- BOM506365
30058
--- BOM532539
30099
--- BOM506390
30140
--- BOM532626
30181
--- BOM521163
30222
--- BOM532761
30263
--- BOM506405
30304
--- BOM532627
30345
--- BOM506480
30385
--- BOM532927
30426
--- BOM506520
30467
--- BOM532443
30508
--- BOM506522
30546
--- BOM506525
30586
--- BOM519156
30627
--- BOM506528
30666
--- BOM515043
30707
--- BOM506530
30714
--- BOM520073
30755
--- BOM506532
30796
--- BOM530001
30837
--- BOM513683
30878
--- BOM506579
30918
--- BOM509496
30959
--- BOM532368
31000
--- BOM506597
31041
--- BOM506605
31066
--- BOM524370
31107
--- BOM506618
31147


--- BOM511730
42908
--- BOM524288
42949
--- BOM526957
42990
--- BOM511734
42994
--- BOM511736
43026
--- BOM532370
43067
--- BOM511738
43068
--- BOM511740
43072
--- BOM532630
43112
--- BOM515147
43153
--- BOM511754
43186
--- BOM511756
43190
--- BOM524174
43231
--- BOM511758
43252
--- BOM532796
43293
--- BOM511760
43298
--- BOM532930
43339
--- BOM511766
43378
--- BOM511768
43398
--- BOM521248
43439
--- BOM526931
43478
--- BOM512018
43509
--- BOM512020
43513
--- BOM519091
43552
--- BOM524774
43593
--- BOM512024
43594
--- BOM526544
43635
--- BOM514448
43675
--- BOM532705
43716
--- BOM512047
43735
--- BOM512048
43756
--- BOM523204
43797
--- BOM519600
43838
--- BOM524669
43879
--- BOM512064
43881
--- BOM524500
43922
--- BOM532654
43963
--- BOM532851
44004
--- BOM512093
44045
--- BOM524764
44086
--- BOM512103
44096
--- BOM512105
44101
--- BOM512109
44102
--- BOM519183
44143
--- BOM512131
44184
--- BOM524520
44225
--- BOM512149
44255
--- BOM533080
44296
--- BOM531266
44337
--- BOM533171
44378


In [9]:
df_backup=df_new_eod.copy()

In [10]:
df_new_eod.head()

,Open,High,Low,Close,WAP,No. of Shares,No. of Trades,Total Turnover,Deliverable Quantity,% Deli. Qty to Traded Qty,Spread H-L,Spread C-O,code
Date,,,,,,,,,,,,,
2019-04-26,1491.10,1491.90,1474.00,1477.50,1487.36,23219.0,401.0,34534935.0,1084.0,4.67,17.9,-13.60,BOM500002
2019-04-30,1478.05,1486.45,1468.45,1473.40,1473.73,27577.0,284.0,40640961.0,987.0,3.58,18.0,-4.65,BOM500002
2019-05-02,1474.00,1519.90,1464.00,1473.90,1493.65,7317.0,682.0,10929019.0,1413.0,19.31,55.9,-0.10,BOM500002
2019-05-03,1474.00,1515.50,1474.00,1490.55,1500.21,4414.0,468.0,6621910.0,1179.0,26.71,41.5,16.55,BOM500002
2019-05-06,1480.00,1503.15,1470.05,1481.30,1481.59,24894.0,491.0,36882619.0,1505.0,6.05,33.1,1.30,BOM500002


In [11]:
import time
time.sleep(8*60)
f = IntProgress(min=0, max=len(sym_date)-quandl_limit) # instantiate the bar
display(f) # display the bar

for sym,last_date in {k: sym_date[k] for k in list(sym_date)[quandl_limit:]}.items():
    f.value += 1
    df_eod=quandl_stocks(symbol=sym,start_date=last_date,mod="BSE")
    df_eod['code']=sym
    if(df_new_eod.empty):
        df_new_eod=df_eod
    else:
        df_new_eod=df_new_eod.append(df_eod)

IntProgress(value=0, max=1846)

C:\ProgramData\Anaconda3\envs\conda36\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [14]:
df_historical_eod=df_historical_eod.append(df_new_eod)

In [15]:
%%time
df_historical_eod.to_pickle("Data/historical_bse.pkl")

Wall time: 23.2 s


In [13]:
df_historical_eod.head()

,% Deli. Qty to Traded Qty,Close,Deliverable Quantity,High,Low,No. of Shares,No. of Trades,Open,Spread C-O,Spread H-L,Total Turnover,WAP,code
Date,,,,,,,,,,,,,
1992-03-02,NaN,575.0,NaN,600,525,0,0,525,50.0,75.0,0.0,0.0,BOM500002
1992-03-03,NaN,625.0,NaN,670,575,0,0,575,50.0,95.0,0.0,0.0,BOM500002
1992-03-10,NaN,650.0,NaN,650,550,0,0,610,40.0,100.0,0.0,0.0,BOM500002
1992-03-11,NaN,600.0,NaN,600,500,0,0,550,50.0,100.0,0.0,0.0,BOM500002
1992-03-12,NaN,550.0,NaN,550,500,0,0,500,50.0,50.0,0.0,0.0,BOM500002


In [4]:
# df_historical_eod_test=pd.read_pickle("historical_bse.pkl")